# As supervised problem 

## Challenge 0

Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. 

Suggested avenues of investigation: 
- Other modeling techniques (SVM?)
- More features that take advantage of the spaCy information (include grammar, phrases, POS, etc)
- Making sentence-level features (number of words, amount of punctuation)
- Including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc)

Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.svm import SVC

# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data (remove chapter lines)
persuasion = gutenberg.raw('austen-persuasion.txt')
persuasion = re.sub(r'Chapter \d+', '', persuasion)
persuasion = text_cleaner(persuasion)

alice = gutenberg.raw('carroll-alice.txt')
alice = re.sub(r'CHAPTER .*', '', alice)
alice = text_cleaner(alice)

emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)

shakespeare = gutenberg.raw('shakespeare-hamlet.txt')
shakespeare = re.sub(r'Actus \w+\.', '', shakespeare)
shakespeare = re.sub(r'Scoena \w+\.', '', shakespeare)
shakespeare = text_cleaner(shakespeare)

In [2]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')

persuasion_doc = nlp(persuasion)
alice_doc = nlp(alice)
emma_doc = nlp(emma)
shakespeare_doc = nlp(shakespeare)

In [3]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]
shakespeare_sents = [[sent, "Shakespeare"] for sent in shakespeare_doc.sents]

# FOR LATER USE
alice_vs_other_sentences = pd.DataFrame(alice_sents + shakespeare_sents)
persuasion_vs_other_sentences = pd.DataFrame(persuasion_sents + shakespeare_sents)
austen_vs_other_sentences = pd.DataFrame(persuasion_sents + emma_sents + shakespeare_sents)

# Combine the sentences from 2 novels into 1 data frame
sentences = pd.DataFrame(persuasion_sents + alice_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [4]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (not token.is_punct and not token.is_stop and token.lemma_ in common_words)]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags
persuasionwords = bag_of_words(persuasion_doc)
alicewords = bag_of_words(alice_doc)

# FOR LATER USE
emmawords = bag_of_words(emma_doc)
shakespearewords = bag_of_words(shakespeare_doc)
common_alice_vs_other_words = set(alicewords + shakespearewords)
common_persuasion_vs_other_words = set(persuasionwords + shakespearewords)
common_austen_vs_other_words = set(persuasionwords + emmawords + shakespearewords)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [ ]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Processing row 1150
Processing row 1200
Processing row 1250
Processing row 1300
Processing row 1350
Processing row 1400
Processing row 1450
Processing row 1500
Processing row 1550
Processing row 1600
Processing row 1650
Processing row 1700
Processing row 1750
Processing row 1800
Processing row 1850


In [ ]:
from sklearn.model_selection import train_test_split

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

First, I will test how an SVM model would do on this same data.

In [ ]:
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

print('Training:', svm.score(X_train, y_train))
print('Test:', svm.score(X_test, y_test))

Next, I will add sentence-level features like # of words, characters, and punctuation, to see if this will improve my model.

In [ ]:
word_counts['# words'] = word_counts['text_sentence'].str.len()
word_counts['# chars'] = word_counts['text_sentence'].apply(lambda x: sum(list(map(lambda y:y.__len__(), x))))

# Utility function to create a list of used punctuation
def bag_of_punctuations(text):
    all_punctuation = [token for token in text if token.is_punct]
    return [item[0] for item in Counter(all_punctuation)]
    
def add_punctuation_features(df):
    for i, sentence in enumerate(df['text_sentence']):
        punctuations = [for token in sentence if token.is_punct]

        for punctuation in punctuations:
            df.loc[i, punctuation] += 1
    
    return df

word_counts = add_punctuation_features(word_counts)
word_counts.head()

In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

svm_with_sent_features = SVC(kernel = 'linear')
svm_with_sent_features.fit(X_train, y_train)

print('Training:', svm_with_sent_features.score(X_train, y_train))
print('Test:', svm_with_sent_features.score(X_test, y_test))

## Challenge 1

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. 

In [ ]:
alice_vs_other_word_counts = bow_features(alice_vs_other_sentences, common_alice_vs_other_words)
alice_vs_other_word_counts.head()

In [ ]:
persuasion_vs_other_word_counts = bow_features(persuasion_vs_other_sentences, common_persuasion_vs_other_words)
persuasion_vs_other_word_counts.head()

In [ ]:
austen_vs_other_word_counts = bow_features(austen_vs_other_sentences, common_austen_vs_other_words)
austen_vs_other_word_counts.head()

In [ ]:
def train_with_SVM(df):
    Y = df['text_source']
    X = np.array(df.drop(['text_sentence','text_source'], 1))
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
    
    svm = SVC(kernel = 'linear')
    svm.fit(X_train, y_train)
    
    print('Training:', svm.score(X_train, y_train))
    print('Test:', svm.score(X_test, y_test))

print('Alice in Wonderland vs. Other Work\n')
print(train_with_SVM(alice_vs_other_word_counts))

print('Persuasion vs. Other Work\n')
print(train_with_SVM(persuasion_vs_other_word_counts))

print('Austen vs. Other Work\n')
print(train_with_SVM(austen_vs_other_word_counts))